In [5]:
import pandas as pd
import numpy as np


# profile report
#from ydata_profiling import ProfileReport

# visualisations
import seaborn as sns
import matplotlib.pyplot as plt

# train test split
from sklearn.model_selection import train_test_split

# label encoder
from sklearn.preprocessing import LabelEncoder

# external functions file
import functions as f


pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

KeyError: 'PolyCollection:kwdoc'

In [ ]:
df = pd.read_csv('./project_data/train_data.csv', index_col = 'Claim Identifier')
df.head(3)

# 2. Exploratory Data Analysis

<a href="#top">Top &#129033;</a>

In [ ]:
df.describe(include='object').T

In [ ]:
df.describe().T

In [ ]:
df.shape

In [ ]:
df.info()

**Correlation matrix**

In [ ]:
# drop column always missing
temp = df.drop('OIICS Nature of Injury Description', axis = 1)

# drop na
temp = temp.dropna()

# select numbers
corr_data = temp.select_dtypes(include=['number'])

correlation_matrix = corr_data.corr(method='spearman') # pearson by default

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='Blues', linewidths=0.1)
plt.show()

**Unique Values**

In [ ]:
for column in df.columns:
    
    if df[column].nunique() < 20:
        
        print(f"Unique values in '{column}': {df[column].unique()}")
        print(df[column].nunique(), '\n')

## 2.2 Incoherencies

<a href="#top">Top &#129033;</a>

**Age at Injury**

very high max values

In [ ]:
df[df['Age at Injury'] > 100]

In [ ]:
f.plot_histogram(df['Age at Injury'], 'Age', 'Frequency',
                'Age Distribution')

**Birth Year**

very Low values for Birth Year, but a LOT of rows with the same issue

In [ ]:
df[df['Birth Year'] < 1800]

In [ ]:
f.plot_histogram(df['Birth Year'], 'Birth Year', 'Frequency',
                'Birth Year Distribution')

**IME-4 Count**

Very High 

In [ ]:
df[df['IME-4 Count'] > 30]

In [ ]:
f.plot_histogram(df['IME-4 Count'], 'IME-4 Count', 'Frequency',
                'IME-4 Count Distribution')

**OIICS Nature of Injury Description**

always missing

In [ ]:
print(f"Number of missing rows in variable OIICS Nature of Injury Description: {len(df[df['OIICS Nature of Injury Description'].isna()])}")
print(f"Number of Rows in the Dataset: {len(df)}")

**Zip Code**

In [ ]:
df[df['Zip Code'].notna() & 
    df['Zip Code'].apply(lambda x: not str(x).isnumeric())]


**Agreement Reached**

w/ a lot of zeros

In [ ]:
df['Agreement Reached'].value_counts()

In [ ]:
f.plot_histogram(df['Agreement Reached'], 'Agreement Reached', 'Frequency',
                'Agreement Reached Distribution')

**WCB Decision**

always the same

In [ ]:
df['WCB Decision'].unique()

**Claim Injury Type**

imbalanced target 

In [ ]:
df['Claim Injury Type'].value_counts()

**Incoherent Columns**

Some columns exist in train data but not on test

In [ ]:
train_columns = set(df.columns)
test_columns = set(test.columns)

not_in_train = train_columns - test_columns
print(f'Columns in train but not in test: {not_in_train}')


## 2.3 Initial Visualisations

<a href="#top">Top &#129033;</a>

# 3. Data Cleaning & Preprocessing

<a href="#top">Top &#129033;</a>

## 3.1 Duplicates

<a href="#top">Top &#129033;</a>

In [ ]:
df[df.duplicated()]

In [ ]:
duplicates = df.duplicated(keep=False)

non_missing_in_other_columns = df.drop(columns=['Assembly Date']).notna().any(axis=1)


df[duplicates & non_missing_in_other_columns]

In [ ]:
df = df.drop_duplicates()

verify the success

In [ ]:
df[df.duplicated()]

## 3.2 Feature Engineering

<a href="#top">Top &#129033;</a>

all transformations must be applied o X_test too

### 3.2.1 Data Type Conversions

even though this is usually feature engineering, it will be useful doing before treating missing values

In [ ]:
#df.info()

In [ ]:
df['Accident Date'] = pd.to_datetime(df['Accident Date'], 
                                     errors='coerce')

df['Assembly Date'] = pd.to_datetime(df['Assembly Date'], 
                                     errors='coerce')

df['C-2 Date'] = pd.to_datetime(df['C-2 Date'], 
                                errors='coerce')

df['C-3 Date'] = pd.to_datetime(df['C-3 Date'], 
                                errors='coerce')

df['First Hearing Date'] = pd.to_datetime(df['First Hearing Date'], 
                                          errors='coerce')

In [ ]:
df.head(2)

### 3.2.2 Encoding

<a href="#top">Top &#129033;</a>

**Alternative Dispute Resolution**

encode ['N' nan 'Y' 'U'] n -> 0, y -> 1, u -> 2

In [ ]:
freq = df['Alternative Dispute Resolution'].value_counts()
freq

In [ ]:
df['Alternative Dispute Resolution'] = df['Alternative Dispute Resolution'].map(freq)

**Attorney/Representative**

['N' 'Y' nan] encode 0/1

In [ ]:
df['Attorney/Representative'].value_counts()

In [ ]:
df['Attorney/Representative'] = df['Attorney/Representative'].replace({'N': 0, 'Y': 1})

**Carrier Name**

In [ ]:
df['Carrier Name'].nunique()

frequency encoder

In [ ]:
freq = df['Carrier Name'].value_counts()

In [ ]:
df['Carrier Name'] = df['Carrier Name'].map(freq)

**Carrier Type**

encode (8 unique values)

In [ ]:
freq = df['Carrier Type'].value_counts()
freq

In [ ]:
df['Carrier Type'] = df['Carrier Type'].map(freq)

**County of Injury**

probably too many to encode (?) --> freq encoding

In [ ]:
df['County of Injury'].nunique()

In [ ]:
freq = df['County of Injury'].value_counts()

In [ ]:
df['County of Injury'] = df['County of Injury'].map(freq)

**COVID-19 Indicator**

binary encoding

In [ ]:
df['COVID-19 Indicator'].value_counts()

In [ ]:
df['COVID-19 Indicator'] = df['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})

**District Name**

encode (8 uniques)

In [ ]:
freq = df['District Name'].value_counts()
freq

In [ ]:
df['District Name'] = df['District Name'].map(freq)

**Gender**

encode ['M' 'F' nan 'U' 'X']

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Gender'] = df['Gender'].map({
    'M': 0,  # Male
    'F': 1,  # Female
    'U': 2,  # Unknown 
    'X': 2   # Other 
})

**Medical Fee Region**

encode

In [ ]:
freq = df['Medical Fee Region'].value_counts()
freq

In [ ]:
df['Medical Fee Region'] = df['Medical Fee Region'].map(freq)

### 3.2.3 Other transformations

<a href="#top">Top &#129033;</a>

**Accident Date**

transform into year (maybe month and day too)

In [ ]:
df['Accident Year'] = df['Accident Date'].dt.year
df['Accident Month'] = df['Accident Date'].dt.month
df['Accident Day'] = df['Accident Date'].dt.day

**Assembly Date**

transform into year (maybe month and day too)

In [ ]:
df['Assembly Year'] = df['Assembly Date'].dt.year
df['Assembly Month'] = df['Assembly Date'].dt.month
df['Assembly Day'] = df['Assembly Date'].dt.day

**C-2 Date**

transform into year (maybe month and day too)

In [ ]:
df['C-2 Year'] = df['C-2 Date'].dt.year
df['C-2 Month'] = df['C-2 Date'].dt.month
df['C-2 Day'] = df['C-2 Date'].dt.day

**Zip Code**

transform non-numeric into missing values

In [ ]:
df['Zip Code'] = df['Zip Code'].apply(lambda x: np.nan 
                                      if not str(x).isnumeric() else x)

**Columns not in test data**

In [ ]:
df = df.drop(['Agreement Reached', 'WCB Decision'], axis = 1)

**Drop Transformed Columns**

In [ ]:
df = df.drop(['Accident Date', 'Assembly Date', 'C-2 Date'], axis = 1)

### 3.2.4 Unique Feature-Pair Analysis 

<a href="#top">Top &#129033;</a>

save codes and descriptions in dataframes, for later consultation (if needed)

In [ ]:
injury_cause = df[['WCIO Cause of Injury Code', 'WCIO Cause of Injury Description']].drop_duplicates()

injury_cause_df = injury_cause.set_index('WCIO Cause of Injury Code')

injury_cause_df.head(2)

In [ ]:
injury_nature = df[['WCIO Nature of Injury Code', 'WCIO Nature of Injury Description']].drop_duplicates()

injury_nature_df = injury_nature.set_index('WCIO Nature of Injury Code')

injury_nature_df.head(2)

In [ ]:
body_code = df[['WCIO Part Of Body Code', 'WCIO Part Of Body Description']].drop_duplicates()

body_code_df = body_code.set_index('WCIO Part Of Body Code')


body_code_df.head(2)

In [ ]:
industry_code = df[['Industry Code', 'Industry Code Description']].drop_duplicates()

industry_code_df = industry_code.set_index('Industry Code')


industry_code_df.head(2)

## 3.3 Missing Values

<a href="#top">Top &#129033;</a>

In [ ]:
df.isna().sum()

In [ ]:
df[df['Claim Injury Type'].isna()]

dropping them

In [ ]:
df.dropna(subset=['Claim Injury Type'], inplace=True)

**Claim Injury Type**

After dropping the missing values we can encode

In [ ]:
encoder = LabelEncoder()
df['Claim Injury Type'] = encoder.fit_transform(df['Claim Injury Type'])
#encoder.inverse_transform([result])

**C-3 Date**

In [ ]:
print(f'There are {len(df[df["C-3 Date"].isna()])} rows with missing values')
df[df['C-3 Date'].isna()].head(2)

In [ ]:
# Create a binary variable: 1 if 'C-3 Date' is not missing, 0 if it is missing
df['C-3 Date Binary'] = df['C-3 Date'].notna().astype(int)

**First Hearing Date**

In [ ]:
print(f'There are {len(df[df["First Hearing Date"].isna()])} rows with missing values')
df[df['First Hearing Date'].isna()].head(2)

In [ ]:
# Create a new variable: 0 if 'First Hearing Date' is missing, otherwise extract the year

df['First Hearing Year'] = df['First Hearing Date'].apply(lambda x: x.year if pd.notna(x) else 0)

**IME-4 Count**

In [ ]:
print(f'There are {len(df[df["IME-4 Count"].isna()])} rows with missing values')
df[df['IME-4 Count'].isna()].head(2)

In [ ]:
df['IME-4 Count'] = df['IME-4 Count'].fillna(0)

**OIICS Nature of Injury Description**

In [ ]:
print(f'There are {len(df[df["OIICS Nature of Injury Description"].isna()])} rows with missing values')
df[df['OIICS Nature of Injury Description'].isna()].head(2)

In [ ]:
# size of missing / size of dataset
len(df[df['OIICS Nature of Injury Description'].isna()]) / len(df)

drop not needed variables 

In [ ]:
df = df.drop(['C-3 Date', 'First Hearing Date', 
             'OIICS Nature of Injury Description'], axis = 1)

# Visualisations

In [ ]:
df

# 3. Export

<a href="#top">Top &#129033;</a>

In [ ]:
df = df.drop(['WCIO Cause of Injury Description', 'WCIO Nature of Injury Description', 
              'WCIO Part Of Body Description', 'Industry Code Description'], axis = 1)

In [ ]:
df.to_csv('./project_data/out_eda1.csv')